<a href="https://colab.research.google.com/github/Daniel-R-Armstrong/geo2vec/blob/master/Project_Maidenhead(s)_by_broadcasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Maidenhead

## Why Maidenhead

When I was looking for a way to measure the relevant distance between an airport and a subject location, I came across an image of an image of a maidenhead map. The goal was to have an easy way for me to find relative distance without a lot of work, oh boy was I wrong. 
Overall there isn’t really anything special about the Maidenhead coordinate system but it was the start of my geospatial journey, it forced me to learn a lot about geospatial data, and helped me to understand and use broadcasting and amazing tool when dealing with tensors of different sizes. 


## Maidenhead Summary

The Maidenhead Locator System (a.k.a. QTH Locator and IARU Locator) is a geographic coordinate system used by amateur radio operators to succinctly describe their locations(wikipedia.org). This system is being used based on the reasonable blocks that the system creates. The hypothesis is that any airport or other location data is not relevant if it is more full grid cell away from the grid cell in question. For example if you open the [Maidenhead Map](http://memeso.net/wp-content/uploads/2018/05/1525151059-latitude-longitude-map-of-us-map-usa-longitude-7-maps-with-latitude-and-on-lines.jpg) Chicago is in cell EN61, it seem logical to me that information in en50,en51,en52, en60, en61,en62, en70 en71,en 72 can all contain information that might be beneficial to understand Chicago. The Maidenhead Locator System has more layers for more precise grid location, but for my use two layers are good starting point.
In the future I want to test a logarithmically weighted based on proximity measurement, that might provide an even better understanding of location, this is based on the thought that the closer you are to an area of interest the more important that the location distance matters. It might be interesting to experiment with a time or population density based system as well.


```
#         ||                          |
# ***2*c  || ...   EN52wc    EN52xc   |   EN62ac    EN62bc     ...
# ***2*b  || ...   EN52wb    EN52xb   |   EN62ab    EN62bb     ...
# ***2*a  || ...   EN52wa    EN52xa   |   EN62aa    EN62ba     ...
# --------||--------------------------|----------------------------
# ***1*x  || ...   EN51wx    EN51xx   |   EN61ax    EN61bx     ...
# ***1*w  || ...   EN51ww    EN51xw   |   EN61aw    EN61bw     ...
# ***1*v  || ...   EN51wv    EN51xv   |   EN61av    EN61bv     ...
#         ||         ...       ...    |    ...       ...       ...
# -----------------------------------------------------------------
#         || ...   **5*w* -> **5*x*  ->   **6*a* -> **6*b*     ...
  
```
helpful websites:
https://en.wikipedia.org/wiki/Maidenhead_Locator_System
http://www.levinecentral.com/ham/grid_square.php

In [0]:
import pandas as pd

# Airports

### Airports - get data

In [0]:
!wget --header="Host: www.udsmapper.org" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.udsmapper.org/zcta-crosswalk.cfm" --header="Cookie: JSESSIONID=3D177A9BA53BF95A133BDEBEF0F0F850.cfusion; __utma=144181233.1974192477.1551962401.1551962401.1551962401.1; __utmc=144181233; __utmz=144181233.1551962401.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); __utmt=1; __utmb=144181233.1.10.1551962401; __zlcmid=rCi4Je0zE9JXNv" --header="Connection: keep-alive" "https://www.udsmapper.org/docs/zip_to_zcta_2018.xlsx" -O "zip_to_zcta_2018.xlsx" -c -nv

2019-07-28 10:51:45 URL:https://www.udsmapper.org/docs/zip_to_zcta_2018.xlsx [1594030/1594030] -> "zip_to_zcta_2018.xlsx" [1]


In [0]:
zcta=pd.read_excel('zip_to_zcta_2018.xlsx')

In [0]:
#I used the CurlWget chrome extention to get the location and infomration need to download the file
!wget --header="Host: simplemaps.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://simplemaps.com/data/us-zips" --header="Cookie: __cfduid=d2eeab3cb7fd5d9b5d2572bc650df34b21547543898; __utma=245557126.1069218867.1547543898.1547543898.1547543898.1; __utmc=245557126; __utmz=245557126.1547543898.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __utmt=1; csrftoken=uUWZtNd1X0R0kob3bwH78KKVHtUddDDq; __utmb=245557126.2.10.1547543898" --header="Connection: keep-alive" "https://simplemaps.com/static/data/us-zips/1.4/uszipsv1.4.csv" -O "uszipsv1.4.csv" -c -nv

2019-07-28 10:52:15 URL:https://simplemaps.com/static/data/us-zips/1.4/uszipsv1.4.csv [5160760/5160760] -> "uszipsv1.4.csv" [1]


In [0]:
#airport codes 
# be careful if using excel, utf-8 issue

# http://ourairports.com/data/airports.csv
# http://ourairports.com/data/
# http://ourairports.com/data/regions.csv
# http://ourairports.com/data/countries.csv
# http://ourairports.com/data/runways.csv

!wget --header="Host: ourairports.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: http://ourairports.com/data/" --header="Cookie: __auc=7b07219f1688f4521993a54a6e3; __utmz=4642349.1548926735.3.2.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); __asc=10aea150168bd2cb5870d9347ed; __utma=4642349.2073409169.1548591899.1548926440.1549362051.4; __utmc=4642349; __utmt=1; __utmb=4642349.1.10.1549362051" --header="Connection: keep-alive" "http://ourairports.com/data/airports.csv" -O "airports.csv" -c -nv

df = pd.read_csv("airports.csv")

#I found it helpful to save the data that you are going to use because you dont know if the website that houses the data is going to break. 
# my saved airport data can be found at: 

#url="https://raw.githubusercontent.com/Daniel-R-Armstrong/Coursera_Capstone/master/airports.csv"
#df = pd.read_csv(url,header=None,low_memory=False)
#headers= list(dfhead)

#you might have to change the data types

2019-07-28 10:52:23 URL:http://ourairports.com/data/airports.csv [8351242/8351242] -> "airports.csv" [1]


### Airports - explore/clean/filter

In [0]:
df.dtypes

In [0]:
df.columns

In [0]:
df.type.unique()

In [0]:
#remove airports with with missing data
df= df.dropna(subset=["municipality","elevation_ft", "type"])

In [0]:
# filter to medium and large airports
df = df[df.type.str.contains('medium_airport|large_airport', regex=True)]

# todo:
#    look into impact of small_airport(?), heliport(hospitals), seaplane_base(costal?), balloonport(?)



In [0]:
#filter out non-us airports 
df = df[(df.iso_country == "US")]

#df1 = df.groupby('product')['value'].sum().reset_index()

In [0]:
# only had to use this when using airport.csv that I stored in github because it read them as objects
df['latitude_deg'] = pd.to_numeric(df['latitude_deg'], errors='coerce')
df['longitude_deg'] = pd.to_numeric(df['longitude_deg'], errors='coerce')
df.dtypes

In [0]:
df.to_csv('med-large_airports.csv', encoding='utf-8', index=False)

In [0]:
ls

#Maidenhead

## Madenhead (lat, log)

In [0]:
# The maidenhead system:
# Encode longitude first, and then latitude.
 
# The first pair (a field) encodes with base 18 and the letters "A" to "R".
    # to avoid negative number uses distance from south pole to northpole and from greenwich eastward (adjusting lat by 90 and log by 180)
    # division of lat is in 10° increments, lon in 20° increments. 
    # size of box aprox. what you would think of as a region
    # converting to unicode with ord() returns int(65-82)
# The second pair (square) encodes with base 10 and the digits "0" to "9".
    # 1° of latitude by 2° of longitude
    # size is a small metropolitan area/large city
# The third pair (subsquare) encodes with base 24 and the letters "a" to "x".
    # size of a city or neighborhood. 
# The fourth pair (extended square) encodes with base 10 and the digits "0" to "9".
    # size of a neighborhood
    
# ord("unicode chr") = int 
# chr(int) = "unicode chr"

def maidenhead(lat, lon):

    A = ord('A')
    a = divmod(lon+180, 20) #divmod(numerator,denominator) returns (quotient and remainder)
    b = divmod(lat+90, 10)
    a_field = chr(A+ int(a[0])) + chr(A+int(b[0])) # a field
    square_lon = divmod(a[1] / 2, 1)
    square_lat = divmod(b[1], 1)
    square = str(int(square_lon[0])) + str(int(square_lat[0]))
    subsquare_lon = divmod(square_lon[1]*24,1)
    subsquare_lat = divmod(square_lat[1]*24,1)
    subsquare = chr(A+int(subsquare_lon[0])).lower() + chr(A+int(subsquare_lat[0])).lower()
    maidenhead = a_field + square + subsquare
    #maidenhead = a_field + square
    return maidenhead
   
    

 Maidenhead (lat, log) - test

In [0]:
m=maidenhead(31.308800,-86.393799) #str
m


'EM61th'

Maidenhead (lat, log) - df.apply

In [0]:
df['maidenhead']=df.apply(lambda x: maidenhead(lat=x['latitude_deg'], lon= x['longitude_deg']), axis=1)
#df['d'] = df.apply(lambda x: maidenhead(a = x['a'], b = x['b'], c = x['c']), axis=1)

In [0]:
df.head(3)

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,maidenhead
6194,12243,5A8,medium_airport,Aleknagik / New Airport,59.2826,-158.617996,66.0,NaN,US,US-AK,Aleknagik,yes,5A8,WKK,5A8,NaN,http://en.wikipedia.org/wiki/Aleknagik_Airport,NaN,BO09qg
25913,19042,K79J,medium_airport,South Alabama Regional At Bill Benton Field Ai...,31.3088,-86.393799,310.0,NaN,US,US-AL,Andalusia/Opp,no,K79J,NaN,79J,NaN,http://en.wikipedia.org/wiki/Andalusia-Opp_Air...,Andalusia Opp Airport,EM61th
26093,3356,KABE,medium_airport,Lehigh Valley International Airport,40.6521,-75.440804,393.0,NaN,US,US-PA,Allentown,yes,KABE,ABE,ABE,NaN,http://en.wikipedia.org/wiki/Lehigh_Valley_Int...,NaN,FN20gp


## Maidenhead (M2Cell, Cell2M)

m_cell creates an integer representation for lat and lon.

In [0]:
def m_cell(lat, lon):
    A = ord('A')
    a = divmod(lon+180, 20) #divmod(numerator,denominator) returns (quotient and remainder)
    b = divmod(lat+90, 10)
    a_field_lon = int(a[0]) 
    a_field_lat = int(b[0])
    a_field = chr(A+ int(a[0])) + chr(A+int(b[0])) # a field
    square_lon_dm = divmod(a[1] / 2, 1)
    square_lat_dm = divmod(b[1], 1)
    square_lon = int(square_lon_dm[0])
    square_lat = int(square_lat_dm[0])
    square = str(int(square_lon_dm[0])) + str(int(square_lat_dm[0]))
    subsquare_lon_dm = divmod(square_lon_dm[1]*24,1)
    subsquare_lat_dm = divmod(square_lat_dm[1]*24,1)
    subsquare_lon = int(subsquare_lon_dm[0])                
    subsquare_lat = int(subsquare_lat_dm[0])
    subsquare = chr(A+int(subsquare_lon_dm[0])).lower() + chr(A+int(subsquare_lat_dm[0])).lower()
    maiden={}
    maiden['maidenhead'] = a_field + square + subsquare
    maiden['m_cell_lon'] = a_field_lon * 18 * 10 * 24 +  square_lon * 10 *24 + subsquare_lon
    maiden['m_cell_lat'] = a_field_lat * 18 * 10 * 24 +  square_lat * 10 *24 + subsquare_lat
    
    return maiden



In [0]:
m=m_cell(31.308800,-86.393799) #str
m

{'m_cell_lat': 52087, 'm_cell_lon': 18739, 'maidenhead': 'EM61th'}

Cell to Maidenhead

In [0]:
def cell2maidenhead(lat_cell, lon_cell):   
    A = ord('A')
    lon_f=divmod(lon_cell,24*10*18)
    lon_s= divmod(lon_f[1],10*24)
    lat_f=divmod(lat_cell,24*10*18)
    lat_s= divmod(lat_f[1],10*24)
    a_field = chr(A+int(lon_f[0])) + chr(A+int( lat_f[0]))
    square = str(int(lon_s[0])) + str(int(lat_s[0]))
    subsquare = chr(A+int(lon_s[1])).lower() + chr(A+int(lat_s[1])).lower()
    maidenhead = a_field + square + subsquare
    
    return maidenhead


In [0]:
cell2maidenhead(52087,18739)

'EM61th'

# Maidenheads

### maidenheadsw (for loop) 

My first attempt to get a list of maidenheads was to first get the sw corner then loop through all the other cells and add them to a list. After learning about numpy and broadcasting I discovered that this type of looping is not the best way to solve the problem, but I wanted to keep the code since I liked writing it. since I was able to get the sw maidenhead, I could have created a loop grabbing all the rest of the required cells, but luckily there is a much better way by using a technique called broadcasting, which makes it possible to use arithmetic on arrays of different sizes.

In [0]:
import math 
#takes lat and long and finds the sw maidenhead using n, as the number of width/height

def maidenheadsw(lat, lon, n):
    num_maidenheads = (n*2+1)**2
    columns = math.sqrt(num_maidenheads)
    rows = math.sqrt(num_maidenheads)

    A = ord('A')
    # field
    field_lon = divmod(lon+180, 20)
    field_lat = divmod(lat+90, 10)
    field = chr(A+ int(field_lon[0])) + chr(A+int(field_lat[0])) 
    
    # Square
    square_lon = divmod(field_lon[1] / 2, 1)
    square_lat = divmod(field_lat[1], 1)
    square = str(int(square_lon[0])) + str(int(square_lat[0]))
    
    # Subsquare
    subsquare_lon = divmod(square_lon[1]*24,1)
    subsquare_lat = divmod(square_lat[1]*24,1)
    subsquare = chr(A+int(subsquare_lon[0])).lower() + chr(A+int(subsquare_lat[0])).lower()
    # Maidenhead
    maidenhead = field + square + subsquare
    # maidenhead = field + square

    subsquare_lat_s = int(subsquare_lat[0]) - n
    #subsquare_lat_n = int(subsquare_lat[0]) + n 
    subsquare_lon_w = int(subsquare_lon[0]) - n
    #subsquare_lon_e = int(subsquare_lon[0]) + n
    ss_lat_cycles = 0
    ss_lon_cycles = 0
    s_lat_cycles = 0
    s_lon_cycles = 0
    field_lat_s = int(field_lat[0])
    field_lon_w = int(field_lon[0])
    square_lat_s= int(square_lat[0])
    square_lon_w= int(square_lon[0])                   

    maiden_sw={}

    # subsquare_lat_s
    
    while subsquare_lat_s < 0:
        ss_lat_cycles += 1
        subsquare_lat_s += 24
    else:
        maiden_sw['subsquare_lat_s']=subsquare_lat_s
        #print(maiden_sw['subsquare_lat_s'])
        #print(ss_lat_cycles)
    
    # subsquare_lon_w
    
    while subsquare_lon_w < 0:
        ss_lon_cycles += 1
        subsquare_lon_w += 24
    else:
        maiden_sw['subsquare_lon_w']=subsquare_lon_w
        #print(maiden_sw['subsquare_lon_w'])
        #print(ss_lon_cycles)
        
   # square_lat_s
    
    while square_lat_s - ss_lat_cycles < 0:
        s_lat_cycles += 1
        square_lat_s  += 10
    else:
        maiden_sw['square_lat_s']= square_lat_s - ss_lat_cycles
        #maiden_sw['s_lat_cycles']=s_lat_cycles
        #print(maiden_sw['square_lat_s'])
        #print(s_lat_cycles)

    # square_lon_w
    
    while square_lon_w - ss_lon_cycles < 0:
        s_lon_cycles += 1
        square_lon_w  += 10
    else:
        maiden_sw['square_lon_w']= square_lon_w - ss_lon_cycles
        #maiden_sw['s_lat_cycles']=s_lat_cycles
        #print(maiden_sw['square_lon_w'])
        #print(s_lon_cycles) 
            
    # field_lat_s 
    
    while field_lat_s - s_lat_cycles < 0:
        field_lat_s += 18
    else:
        maiden_sw['field_lat_s']=field_lat_s - s_lat_cycles
        #print(maiden_sw['field_lat_s'])
        
   # field_lon_w 
    
    while field_lon_w - s_lon_cycles < 0:
        field_lon_w += 18
    else:
        maiden_sw['field_lon_w']=field_lon_w - s_lon_cycles
        #print(maiden_sw['field_lat_s'])  

    return maiden_sw#, columns

In [0]:
msw=maidenheadsw(31.308800,-86.393799, 25);msw


{'field_lat_s': 12,
 'field_lon_w': 4,
 'square_lat_s': 0,
 'square_lon_w': 5,
 'subsquare_lat_s': 6,
 'subsquare_lon_w': 18}

## Using  Numpy and Broadcasting to get list of Maidenheads

### Numpy Basics

The np.arrange function by defalt returns a "rank one array"

```
x = np.arange(5); x.shape
>> (5,)
```
The problem is that rank one arrays dont work I would normally expect:

```
np.array_equal(x,x.T) 
>>True
```
That is because x and its transpose x.T have a shape of =  (5, )
To convert this to a vector you can use the .reshape() function. 

```
x = x.reshape((1,25)) # or x.reshape((1,len(x)))
x, x.shape
>>(array([[0, 1, 2, 3, 4]]), (1, 5))
```
-note* - x has double [[_]]'s now

To avoid using .reshape() all the time you could use np.nweaxis when creating the array
```
col_a= np.arange(cols)[np.newaxis]  # or np.arange(cols).reshape((5,1))
row_a= np.arange(rows)[:, np.newaxis]  # or np.arange(rows).reshape((1,5))

```
It migh be a good habbit to add an assertation statement
```
assert(x.shape == (1,5))
```

#### Broadcasting

#### Visualization of broadcasting:  



<img src="https://jakevdp.github.io/PythonDataScienceHandbook/figures/02.05-broadcasting.png" alt="Broadcasting Visual"> 
  


The plan


We are going to use Numpy to create a matrix of the adjustments to maidenhead, which will be used to create a list of maidenheads. 
n is the number of cells/maidenheads on each side of the subject maidenhead. an n=2 would give us 2 on each side of the subject, or 25 cells/maidenheads in total. 
```
            *  *  *  *  *
            *  *  *  *  *
            *  *  S  *  *
            *  *  *  *  *
            *  *  *  *  *

```



Maidenheads follow a pattern

```
#         ||                          |
# ***2*c  || ...   EN52wc    EN52xc   |   EN62ac    EN62bc     ...
# ***2*b  || ...   EN52wb    EN52xb   |   EN62ab    EN62bb     ...
# ***2*a  || ...   EN52wa    EN52xa   |   EN62aa    EN62ba     ...
# --------||--------------------------|----------------------------
# ***1*x  || ...   EN51wx    EN51xx   |   EN61ax    EN61bx     ...
# ***1*w  || ...   EN51ww    EN51xw   |   EN61aw    EN61bw     ...
# ***1*v  || ...   EN51wv    EN51xv   |   EN61av    EN61bv     ...
#         ||         ...       ...    |    ...       ...       ...
# -----------------------------------------------------------------
#         || ...   **5*w* -> **5*x*  ->   **6*a* -> **6*b*     ...
  
```
what we need to do is make two differnt arrays that can be used for the adjustments that need to be made.


we want to build two different arrays so that we can use broadcasting to get a list of all the maindenheads

```
array([[ 2,  2,  2,  2,  2],
       [ 1,  1,  1,  1,  1],
       [ 0,  0,  0,  0,  0],
       [-1, -1, -1, -1, -1],
       [-2, -2, -2, -2, -2]])
       
```
And
```
array([[-2, -1,  0,  1,  2],
       [-2, -1,  0,  1,  2],
       [-2, -1,  0,  1,  2],
       [-2, -1,  0,  1,  2],
       [-2, -1,  0,  1,  2]])

```


### Set-up  : matix or numpy.ndarray. 



A Matrix is a speical type of ndarray

In [0]:
n=2
num_maidenheads = (n*2+1)**2
n_rows = n_cols = int(math.sqrt(num_maidenheads))

Create a n_row by n_cols array of all ones

In [0]:
import numpy as np
x = np.ones((n_rows,n_cols), dtype=np.int);x, x.shape

(array([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]]), (5, 5))

Rows and columns

In [0]:
row_a= np.arange(n_rows)[np.newaxis] # or np.arange(row_a).reshape((1,5))
col_a= np.arange(n_cols)[:, np.newaxis] # or np.arange(col_a).reshape((5,1))

In [0]:
col_a, col_a.shape

(array([[0],
        [1],
        [2],
        [3],
        [4]]), (5, 1))

In [0]:
cn=col_a[::-1]-n;cn      


#Equivalent to m[::-1,...]. Does not require the array to be two-dimensional
#The dots (...) represent as many colons as needed to produce a complete indexing tuple

array([[ 2],
       [ 1],
       [ 0],
       [-1],
       [-2]])

In [0]:
lat_adj= cn * x; lat_adj

array([[ 2,  2,  2,  2,  2],
       [ 1,  1,  1,  1,  1],
       [ 0,  0,  0,  0,  0],
       [-1, -1, -1, -1, -1],
       [-2, -2, -2, -2, -2]])

In [0]:
row_a, row_a.shape

(array([[0, 1, 2, 3, 4]]), (1, 5))

In [0]:
rn= row_a -n; rn

array([[-2, -1,  0,  1,  2]])

In [0]:
lon_adj= rn * x; lon_adj

array([[-2, -1,  0,  1,  2],
       [-2, -1,  0,  1,  2],
       [-2, -1,  0,  1,  2],
       [-2, -1,  0,  1,  2],
       [-2, -1,  0,  1,  2]])

## Maidenheads function (parts)

Lat and Lon

In [0]:
lat,lon = 31.308800,-86.393799

Maidenhead and cell pieces:

In [0]:
A = ord('A')
a = divmod(lon+180, 20) #divmod(numerator,denominator) returns (quotient and remainder)
b = divmod(lat+90, 10)
a_field_lon = int(a[0]) 
a_field_lat = int(b[0])
#a_field = chr(A+ int(a[0])) + chr(A+int(b[0])) # a field
square_lon_dm = divmod(a[1] / 2, 1)
square_lat_dm = divmod(b[1], 1)
square_lon = int(square_lon_dm[0])
square_lat = int(square_lat_dm[0])
#square = str(int(square_lon_dm[0])) + str(int(square_lat_dm[0]))
subsquare_lon_dm = divmod(square_lon_dm[1]*24,1)
subsquare_lat_dm = divmod(square_lat_dm[1]*24,1)
subsquare_lon = int(subsquare_lon_dm[0])                
subsquare_lat = int(subsquare_lat_dm[0])
#subsquare = chr(A+int(subsquare_lon_dm[0])).lower() + chr(A+int(subsquare_lat_dm[0])).lower()
    
#  get cell 
m_cell_lon = a_field_lon * 18 * 10 * 24 +  square_lon * 10 *24 + subsquare_lon
m_cell_lat = a_field_lat * 18 * 10 * 24 +  square_lat * 10 *24 + subsquare_lat
    

Broadcasting allows you to add the subject cell value to the lattatue and longatude adjustment 2-d array

In [0]:
lat_array = lat_adj + m_cell_lat
lon_array = lon_adj + m_cell_lon
lat_array, lon_array

(array([[52089, 52089, 52089, 52089, 52089],
        [52088, 52088, 52088, 52088, 52088],
        [52087, 52087, 52087, 52087, 52087],
        [52086, 52086, 52086, 52086, 52086],
        [52085, 52085, 52085, 52085, 52085]]),
 array([[18737, 18738, 18739, 18740, 18741],
        [18737, 18738, 18739, 18740, 18741],
        [18737, 18738, 18739, 18740, 18741],
        [18737, 18738, 18739, 18740, 18741],
        [18737, 18738, 18739, 18740, 18741]]))

We have already created a fuction(cell2maidenhead) that can convert a cell to a maidenhead now we need to be able to apply it to the array of cells that we have created by using the np.vectorize function. From my understanding this still uses a loop so we might want to refactor this function.

In [0]:
np_cell2maidenhead = np.vectorize(cell2maidenhead)

In [0]:
np_maids=np_cell2maidenhead(lat_array,lon_array)
np_maids

array([['EM61rj', 'EM61sj', 'EM61tj', 'EM61uj', 'EM61vj'],
       ['EM61ri', 'EM61si', 'EM61ti', 'EM61ui', 'EM61vi'],
       ['EM61rh', 'EM61sh', 'EM61th', 'EM61uh', 'EM61vh'],
       ['EM61rg', 'EM61sg', 'EM61tg', 'EM61ug', 'EM61vg'],
       ['EM61rf', 'EM61sf', 'EM61tf', 'EM61uf', 'EM61vf']], dtype='<U6')

Great it looks like we have the maidenheads that we want, now we need to convert them to a list. we can use the np.tolist() to convert to a list.

In [0]:
l=np_maids.tolist();l

[['EM61rj', 'EM61sj', 'EM61tj', 'EM61uj', 'EM61vj'],
 ['EM61ri', 'EM61si', 'EM61ti', 'EM61ui', 'EM61vi'],
 ['EM61rh', 'EM61sh', 'EM61th', 'EM61uh', 'EM61vh'],
 ['EM61rg', 'EM61sg', 'EM61tg', 'EM61ug', 'EM61vg'],
 ['EM61rf', 'EM61sf', 'EM61tf', 'EM61uf', 'EM61vf']]

The tolist() function returned a list but it is a nested list(list of lists), look at the extra square brackets, so we are going to have to flatten it. I choose to flatten before the cell2maidenhead function. We can flatten it with the np.flatten() function.

In [0]:
f_lat = lat_array.flatten()
f_lon = lon_array.flatten()
f_lat,f_lon

(array([52089, 52089, 52089, 52089, 52089, 52088, 52088, 52088, 52088,
        52088, 52087, 52087, 52087, 52087, 52087, 52086, 52086, 52086,
        52086, 52086, 52085, 52085, 52085, 52085, 52085]),
 array([18737, 18738, 18739, 18740, 18741, 18737, 18738, 18739, 18740,
        18741, 18737, 18738, 18739, 18740, 18741, 18737, 18738, 18739,
        18740, 18741, 18737, 18738, 18739, 18740, 18741]))

Convert cells to maidenheads with flattened arrays

In [0]:
maidenhead_array = np_cell2maidenhead(f_lat,f_lon)

Convert flattened maidenhead array to list

In [0]:
maidenhead_list = maidenhead_array.tolist()
maidenhead_list


['EM61rj',
 'EM61sj',
 'EM61tj',
 'EM61uj',
 'EM61vj',
 'EM61ri',
 'EM61si',
 'EM61ti',
 'EM61ui',
 'EM61vi',
 'EM61rh',
 'EM61sh',
 'EM61th',
 'EM61uh',
 'EM61vh',
 'EM61rg',
 'EM61sg',
 'EM61tg',
 'EM61ug',
 'EM61vg',
 'EM61rf',
 'EM61sf',
 'EM61tf',
 'EM61uf',
 'EM61vf']

## Maidenheads - function (full)

In [0]:
def cell2maidenhead(lat_cell, lon_cell):
    
    A = ord('A')
    
    lon_f=divmod(lon_cell,24*10*18)
    lon_s= divmod(lon_f[1],10*24)

    lat_f=divmod(lat_cell,24*10*18)
    lat_s= divmod(lat_f[1],10*24)
    
    a_field = chr(A+int(lon_f[0])) + chr(A+int( lat_f[0]))
    square = str(int(lon_s[0])) + str(int(lat_s[0]))
    subsquare = chr(A+int(lon_s[1])).lower() + chr(A+int(lat_s[1])).lower()
    maidenhead = a_field + square + subsquare
    
    return maidenhead

In [0]:
def maidenheads(lat, lon, n=0):
    
    num_maidenheads = (n*2+1)**2
    n_rows = n_cols = int(math.sqrt(num_maidenheads))
    #  setup arrays
    x = np.ones((n_rows,n_cols), dtype=np.int)
    row_a= np.arange(n_rows)[np.newaxis]
    col_a= np.arange(n_cols)[:, np.newaxis]
    cn=col_a[::-1]-n
    lat_adj = cn * x
    rn= row_a - n
    lon_adj = rn * x
    #  get subject maidenhead from lat,lon
    A = ord('A')
    a = divmod(lon+180, 20) #divmod(numerator,denominator) returns (quotient and remainder)
    b = divmod(lat+90, 10)
    a_field_lon = int(a[0]) 
    a_field_lat = int(b[0])
    #a_field = chr(A+ int(a[0])) + chr(A+int(b[0])) # a field
    square_lon_dm = divmod(a[1] / 2, 1)
    square_lat_dm = divmod(b[1], 1)
    square_lon = int(square_lon_dm[0])
    square_lat = int(square_lat_dm[0])
    #square = str(int(square_lon_dm[0])) + str(int(square_lat_dm[0]))
    subsquare_lon_dm = divmod(square_lon_dm[1]*24,1)
    subsquare_lat_dm = divmod(square_lat_dm[1]*24,1)
    subsquare_lon = int(subsquare_lon_dm[0])                
    subsquare_lat = int(subsquare_lat_dm[0])
    #subsquare = chr(A+int(subsquare_lon_dm[0])).lower() + chr(A+int(subsquare_lat_dm[0])).lower()
    
    #  get cell 
    m_cell_lon = a_field_lon * 18 * 10 * 24 +  square_lon * 10 *24 + subsquare_lon
    m_cell_lat = a_field_lat * 18 * 10 * 24 +  square_lat * 10 *24 + subsquare_lat
    
    # cell arrays
    lat_array= m_cell_lat + lat_adj 
    lon_array= m_cell_lon + lon_adj
    
    #flatten
    f_lat = lat_array.flatten()
    f_lon = lon_array.flatten()

    #convert to maidenhead
    np_cell2maidenhead = np.vectorize(cell2maidenhead)    
    maidenhead_array = np_cell2maidenhead(f_lat,f_lon)
        
    #convert maidenheads to list
    maidenhead_list = maidenhead_array.tolist()
    
    return maidenhead_list

    
    

In [0]:
t=maidenheads(31.308800,-86.393799,2)
t

['EM61rj',
 'EM61sj',
 'EM61tj',
 'EM61uj',
 'EM61vj',
 'EM61ri',
 'EM61si',
 'EM61ti',
 'EM61ui',
 'EM61vi',
 'EM61rh',
 'EM61sh',
 'EM61th',
 'EM61uh',
 'EM61vh',
 'EM61rg',
 'EM61sg',
 'EM61tg',
 'EM61ug',
 'EM61vg',
 'EM61rf',
 'EM61sf',
 'EM61tf',
 'EM61uf',
 'EM61vf']

In [0]:
n=2
df['maidenheads']=df.apply(lambda x: maidenheads(lat=x['latitude_deg'], lon= x['longitude_deg'], n=n), axis=1)

In [0]:
df.head(3)

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,maidenhead,maidenheads
6194,12243,5A8,medium_airport,Aleknagik / New Airport,59.2826,-158.617996,66.0,NaN,US,US-AK,Aleknagik,yes,5A8,WKK,5A8,NaN,http://en.wikipedia.org/wiki/Aleknagik_Airport,NaN,BO09qg,"[BO09oi, BO09pi, BO09qi, BO09ri, BO09si, BO09o..."
25913,19042,K79J,medium_airport,South Alabama Regional At Bill Benton Field Ai...,31.3088,-86.393799,310.0,NaN,US,US-AL,Andalusia/Opp,no,K79J,NaN,79J,NaN,http://en.wikipedia.org/wiki/Andalusia-Opp_Air...,Andalusia Opp Airport,EM61th,"[EM61rj, EM61sj, EM61tj, EM61uj, EM61vj, EM61r..."
26093,3356,KABE,medium_airport,Lehigh Valley International Airport,40.6521,-75.440804,393.0,NaN,US,US-PA,Allentown,yes,KABE,ABE,ABE,NaN,http://en.wikipedia.org/wiki/Lehigh_Valley_Int...,NaN,FN20gp,"[FN20er, FN20fr, FN20gr, FN20hr, FN20ir, FN20e..."
